ปฏิบัติการครั้งที่ 7 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

1. ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้นบันทึกไว้ในไฟล์ *.pdf

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้  
http://www.donlapark.cmustat.com/229351/data/AirPassengers.csv  

In [ ]:
!pip install wget

!python -m wget -o AirPassengers.csv http://www.donlapark.cmustat.com/229351/data/AirPassengers.csv

In [ ]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
data = pd.read_csv('AirPassengers.csv')

print(data.head())

print (data.dtypes)

In [ ]:
# ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
def parser(dates):
  return datetime.strptime(dates, '%Y-%m')

parser("2022-02")

###ในการสร้าง time series นั้น เราต้องกำหนดให้ Month เป็น index

In [ ]:
# parse_dates ชื่อของคอลัมน์ที่จะเปลี่ยนให้เป็น datetime
# index_col ชื่อของคอลัมน์ที่จะให้เป็น index
# date_parser ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
data = pd.read_csv('AirPassengers.csv', parse_dates=['Month'], 
                                        index_col='Month',
                                        date_parser=parser)

print (data.head())

print (data.dtypes)

In [ ]:
#1. Specific the index as a string constant:
ts = data['#Passengers']

print(ts)

In [ ]:
ts['1950']

In [ ]:
plt.plot(ts);

###จากแผนภาพข้างล่างนี้จะเห็นว่าความแปรปรวนเพิ่มขึ้นตามเวลา

In [ ]:
roll_std = ts.rolling(window=12).std()
plt.plot(roll_std, label = 'Rolling Std');

###ทดสอบด้วย KPSS test และ ADF test

KPSS
- $H_0$: $\texttt{ts}$ เป็น stationary
- $H_1$: $\texttt{ts}$ มี unit root

ADF
- $H_0$: $\texttt{ts}$ มี unit root
- $H_1$: $\texttt{ts}$ เป็น stationary


###ถ้าผลที่ได้จากการทดสอบสองตัวนี้ไม่สอดคล้องกัน แสดงว่ามีข้อมูลไม่เพียงพอ

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

def KPSStest(timeseries):
  kpsstest = kpss(timeseries, lags='auto')
  print('KPSS test')
  print('Test Statistic: ',kpsstest[0])
  print('p-value: ',kpsstest[1])
  print('Number of Observations Used: ',kpsstest[2])
  print('Critical Values: ',kpsstest[3])

def ADFtest(timeseries):
  adftest = adfuller(timeseries)
  print('ADF test')
  print('Test Statistic: ',adftest[0])
  print('p-value: ',adftest[1])
  print('#Lags Used: ',adftest[2])
  print('Number of Observations Used: ',adftest[3])
  print('Critical Values: ',adftest[4])


KPSStest(ts)
ADFtest(ts)

#### ปรับความแปรปรวนด้วยการใส่ log

In [ ]:
ts_log = np.log(ts)
plt.plot(ts_log)

In [ ]:
roll_std = ts_log.rolling(window=12).std()
plt.plot(roll_std, label = 'Rolling Std');

In [ ]:
KPSStest(ts_log)
ADFtest(ts_log)

### ทำ differencing โดยใช้ $\texttt{time_series.shift()}$ ในการเลื่อนข้อมูลไปข้างหน้า 1 วัน

In [ ]:
ts_log_diff = ts_log - ts_log.shift() #y_(t)-y_(t-1)

plt.plot(ts_log_diff);

In [ ]:
print(ts_log)

print(ts_log.shift())


In [ ]:
KPSStest(ts_log_diff)
ADFtest(ts_log_diff)

### เกิด Error เพราะว่าข้อมูลแถวแรกเป็น NaN ซึ่งเกิดจากการเลื่อนข้อมูลไปข้างหน้า 1 วัน

In [ ]:
ts_log_diff

In [ ]:
ts_log_diff.dropna(inplace=True)
ts_log_diff

In [ ]:
KPSStest(ts_log_diff)
ADFtest(ts_log_diff)

###ลอง differencing อีกหนึ่งครั้งถ้ายังไม่พอใจกับผลที่ได้

In [ ]:
ts_log_diff2 = ts_log_diff - ts_log_diff.shift()
ts_log_diff2.dropna(inplace=True)
ts_log_diff2

plt.plot(ts_log_diff2);

In [ ]:
KPSStest(ts_log_diff2)
ADFtest(ts_log_diff2)

###สร้างแผนภาพ ACF และ PACF

- ค่า ACF ที่ลดลงไปเรื่อยๆ แสดงถึง trend
- จุดที่ peak ขึ้นมาใน ACF แสดงถึง seasonality
- จุดที่เกินบริเวณสีฟ้าใน ACF แสดงถึง lag ที่ช่วยในการทำนาย $y_t$
- ใช้ PACF เพื่อดูว่า lag ไหนที่ไม่เกี่ยวข้องกันบ้าง ถ้าค่า PACF เกินบริเวณสีฟ้าแสดงว่า lag นั้นไม่เกี่ยวข้องกับ lag ก่อนหน้า

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, ax = plt.subplots(2,1,figsize=(10, 10))
plot_acf(ts, lags=25, ax=ax[0]);
plot_pacf(ts, lags=25, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10, 10))
plot_acf(ts_log_diff2, lags=18, ax=ax[0]);
plot_pacf(ts_log_diff2, lags=18, ax=ax[1]);